# COVID Testing and Vaccines and Health Equity 

#### **Analysis Goal**
Highest burden of COVID-19 as of May 2021*: 
1. Pelham Gardens, Bronx, 10469: Station Gun Hill Road (5)
2. Corona, Queens, 11368: Station 103 St-Corona Plaza (7)
3. Midwood, Brooklyn, 11230: Station Avenue M (BQ)

Source: https://www.nytimes.com/interactive/2020/nyregion/new-york-city-coronavirus-cases.html?#zipcode<br/>
*New York Times stoppped reporting COVID-19 cases by zipcodes on May 25, 2021. 


Corona, New York was one of the zip codes ([11368](https://goo.gl/maps/wCcrT4X5FKj79vES7)) with [very high cases](https://www1.nyc.gov/site/doh/covid/covid-19-data-neighborhoods.page) of COVID-19. Its residents are mostly Hispanic and [low-income](https://censusreporter.org/profiles/86000US11368-11368/), which this analysis uses as a proxy for essential workers during the coronavirus pandemic.<br/>

The New York City Department of Health and Mental Hygiene (NYCDOH) has [mobile testing units](https://www1.nyc.gov/site/coronavirus/get-tested/covid-19-testing.page) which often set up near subway stations. However, the testing units operate during peak hours (8 AM - 8 PM), and off-peak commuters cannot access these services. The results of this analysis could provide NYCDOH with evidence to deploy units during off-peak hours to stations in zip codes with high numbers of essential workers. 

#### **Process**
This analysis explores April 2019, off-peak travel on the 7 train (12 AM - 4 AM) from the 103rd Street-Corona station, in the Corona neighborhood in Queens. MTA turnstile data for 2020 and 2021 is irregular because the MTA suspended 24-hour subway service from [May 2020](https://www.nytimes.com/2020/05/06/nyregion/nyc-subway-close-coronavirus.html) until [May 2021](https://www.nytimes.com/2021/05/17/nyregion/nyc-subway-full-service-24-hours.html). Therefore, the best approximation for off-hour travel was the 2019 data used here. 

#### **Preliminary visualization**
The two barplots show how many people entered the Corona station each day in April during the midnight and 4 AM turnstile report.   

#### **Preliminary conclusions**
The barplots show regular off-peak travel and suggest some days of the week may be the best option for stationing mobile testing units late at night. 


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = "{:,.0f}".format


In [ ]:
# Load data from April, May, and June 2019, data is loaded on Saturdays.
# Source: http://web.mta.info/developers/turnstile.html

def get_data_parse_dt(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], usecols = [0,1,2,3,4,6,7,9], keep_date_col=True))
    return pd.concat(dfs)

# Create a datetime column using data from columns DATE and TIME during import
week_nums = [190406, 190413, 190420, 190427, 190504, 190511, 190518, 190525, 190601, 190608, 190615, 190622, 190629, 190706]
turnstiles_df = get_data_parse_dt(week_nums)
turnstiles_df.tail()


In [ ]:
turnstiles_df.columns

In [ ]:
# Strip whitespace from column names
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

In [ ]:
turnstiles_df.head(1)

In [ ]:
# Looking for station of interest: Elmhurst Avenue, Queens 
df = turnstiles_df[turnstiles_df['STATION'].str.contains('CORONA', na=False)] 
df.head(1)
# Station name as recorded in dataset: 103 ST-CORONA

In [ ]:
# Looking for station of interest: Gun Hill Road, Bronx 
df = turnstiles_df[turnstiles_df['STATION'].str.contains('GUN HILL', na=False)] 
df.head(1)
# Station name as recorded in dataset: GUN HILL RD

In [ ]:
# Looking for station of interest: Avenue M, Brooklyn 
df = turnstiles_df[turnstiles_df['STATION'].str.contains('GUN HILL', na=False)] 
df.head(1)
# Station name as recorded in dataset: AVENUE M

In [ ]:
# Mask selects stations of interest 
station_mask = ((turnstiles_df['STATION'] == '103 ST-CORONA') | 
                (turnstiles_df['STATION'] == 'GUN HILL RD') | 
                (turnstiles_df['STATION'] == 'AVENUE M')) 

In [ ]:
station_df = turnstiles_df[station_mask]
station_df.STATION.unique()
# confirm station_df has data from three stations of interest

In [ ]:
# Mask selects dates of interest; include last day in March to calculate the previous day's entries
date_mask = ((station_df['DATE'] >= '03/31/2019') & 
             (station_df['DATE'] <= '06/30/2019'))

In [ ]:
station_date_df = station_df[date_mask]
station_date_df.DATE.unique()
# comfirm station_date_df has dates from March 31 – June 30, 2019

In [ ]:
# Mask selects the interval between 12:00 AM and 4:00 AM from the selected station data df
time_mask = (station_date_df['TIME'] == '04:00:00')

In [ ]:
station_date_time_df = station_date_df[time_mask]
station_date_time_df.TIME.unique()
# confirm station_time_df has data from the 4:00 AM interval

In [ ]:
# station_date_time_df has 2,380 entries with no null values, and data_time column is a dt datatype
station_date_time_df.info()

In [ ]:
highest_burden_df = station_date_time_df.reset_index()
highest_burden_df.head()

In [ ]:
# highest_burden_daily_df = (highest_burden_df
#                            .groupby(["STATION", "DATE"])[['DAILY_ENTRIES']]
#                            .sum().reset_index())

turnstiles = (highest_burden_df.groupby(['STATION'])['SCP'].count())
turnstiles

In [ ]:
# Create new columns for the previous date and entries. 
# Apply a shift to calculate the previous day's entries. 

highest_burden_df[['PREV_DATE', 'PREV_ENTRIES']] = (highest_burden_df
                                                  .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                  .apply(lambda grp: grp.shift(1)))
highest_burden_df.head(2)

In [ ]:
# Drop row with the March 31, 2019 data, used it to calculate the previous entries, no longer neeeded. 
highest_burden_df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
highest_burden_df.head(2)

In [ ]:
# How many stations have a counter going in reverse? 
(highest_burden_df[highest_burden_df["ENTRIES"] < highest_burden_df["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

In [ ]:
# Adjust counter before calculating daily entries
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        return 0
    return counter

highest_burden_df["LATE_NIGHT_ENTRIES"] = highest_burden_df.apply(get_daily_counts, axis=1, max_counter=10_00_000)
highest_burden_df.head(40)
highest_burden_df.sort_values('DATE')

In [ ]:
# Sum all turnstiles per station
highest_burden_daily_df = (highest_burden_df
                           .groupby(["STATION", "DATE"])[['LATE_NIGHT_ENTRIES']]
                           .sum().reset_index())
highest_burden_daily_df.head()

# Reduces dataframe from 2,380 to 271 rows as expected (3 stations * 90 days per station)

In [ ]:
# Add day of week and week number to dataframe in new columns
highest_burden_daily_df['DAY_OF_WEEK_NUM'] = pd.to_datetime(highest_burden_daily_df['DATE']).dt.dayofweek
# highest_burden_daily_df['WEEK_OF_YEAR'] = pd.to_datetime(highest_burden_daily_df['DATE']).dt.isocalendar().week
highest_burden_daily_df.head(20)

In [ ]:
# Add column ROLLING_MEAN to capture the rolling mean (by 7) for each station group
highest_burden_daily_df['ROLLING_MEAN'] = (highest_burden_daily_df
                                           .groupby('STATION')['LATE_NIGHT_ENTRIES']
                                           .transform(lambda x: x.rolling(7,1).mean()))
highest_burden_daily_df.head(2)                    

In [ ]:
# calculate WEEK_DAY_MEAN (no column to capture data, multi-index)
weekly_averages = (highest_burden_daily_df
               .groupby(['STATION','DAY_OF_WEEK_NUM'])['LATE_NIGHT_ENTRIES']
               .mean())
weekly_averages.head(20)

In [ ]:
weekly_averages.unstack(level=0).plot(kind='bar', 
                                      subplots=False, 
                                      rot=0, 
                                      figsize=(15, 5), 
                                      layout=(1, 3), 
                                      xlabel=('Day of the week'),
                                      ylabel=('Average number of entries'),
                                      title=('Average late night entries by day of week at Avenue M, Elmhurst, and Gun Hill Road')
                                     );
                                 
xpositions = (0,1,2,3,4,5,6)
xlabels = ('Mon', 'Tue','Wed','Thu', 'Fri', 'Sat', 'Sun')
plt.xticks(xpositions, xlabels)

ypositions = (0,5000, 10000, 15000, 20000, 25000, 30000)
ylabels = ('0','5,000', '10,000', '15,000', '20,000', '25,000', '30,000')
plt.yticks(ypositions, ylabels)

plt.grid();

